In [ ]:
#install pyod
!pip install pyod

     |████████████████████████████████| 111 kB 5.4 MB/s 
  Created wheel for pyod: filename=pyod-0.9.4-py3-none-any.whl size=130726 sha256=b4a8760fdc17e0b6336e785ff73ac9ff2e1adde70a8d8cfcba745c6881d6bc38
  Stored in directory: /root/.cache/pip/wheels/50/32/84/d64079e8bf087c1b231c9beecfa9abc08592ab5f5f333124cd
Successfully built pyod


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import warnings
warnings.simplefilter("ignore")

cites: https://towardsdatascience.com/a-simple-way-to-detect-anomaly-3d5a48c0dae0

---
Load the data

---

In [8]:
df  = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


---
get features and label for each instance.

check numbers of fraud and not fraud.

---

In [9]:
X =df.drop('Class',1)
y =df['Class']
print('length fraud: ',len(y[y==1]))
print('length not fraud: ',len(y[y==0]))

length fraud:  492
length not fraud:  284315


--- 
as we see the data is extremely imbalanced so 
We will assume the observations in the main class (not fraud) as normal data, and use only these data to train our model. Then, we are able to predict whether a new observation is normal.
now we can say this is a novelty detection task. 

**outlier detection:** The training data contains outliers which are defined as observations that are far from the others. 

**novelty detection:** The training data is not polluted by outliers and we are interested in detecting whether a new observation is an outlier. 

---

Gaussian mixture models are a probabilistic model for representing normally distributed cluster within an overall population. Mixture models in general don't require knowing which cluster a data point belongs to, allowing the model to learn the cluster automatically.

Gaussian Mixture Models, or simply GMMs, attempt to using a soft clustering instead of a hard one.

---


In [11]:
#split data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

---
get all data that in normal class to train GMM on it to understand its distribution 

---

In [12]:
#main_cluster_x contain all features for all train points with y= 0 (normal class)
#main_cluster_y contain all labels for all train points in normal class
main_cluster_y=y_train[y_train==0]
main_cluster_x=X_train[y_train==0]

---
Train GMM with one component, as we have only one class.
then compute score_samples that return log-likelihood of each sample in training data.

---

In [13]:
gmm = GaussianMixture(n_components=1)
gmm.fit(main_cluster_x)
OKscore = gmm.score_samples(main_cluster_x)

---
calculate threshold from main data score.

choose num_std by try and error. 

---

In [30]:
def get_thred(score,num_std=1):
  '''input: log-likelihood , number of std.
     output: threshold.'''
  return score.mean() - num_std*score.std() 


thred = get_thred(OKscore)

---
get log-likelihood of each sample in test data.

---

In [31]:
scores=gmm.score_samples(X_test)

---
calculate y_predict from threshold

---

In [32]:
y_pred=(scores<thred).astype(int)

---
measure accuracy with area under the curve metric

---

In [33]:
fp, tp, _ = metrics.roc_curve(y_test, y_pred)
metrics.auc(fp, tp)

0.9134616047645219

## supervised approach
LogisticRegression

In [28]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
lr_y_pred=lr.predict(X_test)

In [29]:
fp, tp, _ = metrics.roc_curve(y_test, lr_y_pred)
metrics.auc(fp, tp)

0.8490302585333663

## Pyod

In [ ]:
# train the COPOD detector
from pyod.models.copod import COPOD
clf = COPOD()
clf.fit(X_train)

COPOD(contamination=0.1, n_jobs=1)

In [ ]:
COPOD_fpr, COPOD_tpr, _ = metrics.roc_curve(y_test, clf.predict(X_test))
metrics.auc(COPOD_fpr, COPOD_tpr)

0.9026026844911009